# Implementation - Graphical model

In [1]:
import numpy  as np
import igraph as ig
import pyvis.network as net
%run ./2\ Implementation\ -\ Factors\ and\ operations.ipynb

# 1 Factor graph

### 1.1 Factor graph data structure

In [2]:
class factor_graph:
    def __init__(self):
        self._graph = ig.Graph()
    
    # ----------------------- Factor node actions -----------
    def add_factor_node(self, name, factor_):
        if self.__check_variable(name, exception=True) != False:
            raise Exception('Invalid name')
        if type(factor_) is not factor:
            raise Exception('Data is not factor')
        if name in factor_.get_variables():
            raise Exception('Invalid factor name')
            
        for var_name in factor_.get_variables():
            if self.__check_variable(var_name) == False:
                self.__create_variable_node(var_name)
            
        self._graph.add_vertex(name)
        self._graph.vs.find(name=name)['is_factor']   = True
        self._graph.vs.find(name=name)['factor_node'] = factor_
            
        start = self._graph.vs.find(name=name).index
        edge_list = [tuple([start, self._graph.vs.find(name=i).index]) for i in factor_.get_variables()]
        self._graph.add_edges(edge_list)
        
    #def change_factor(self, name, factor_, remove_zero_deg_var=False):
        
    def remove_factor(self, name, remove_zero_deg_var=False):
        self.__check_variable(name, exception=True, target='factor')
        
        factor_neighbors = self._graph.neighbors(name, mode="out")
        g.delete_vertices(name)
        
        if remove_zero_deg_var:
            for var_name in factor_neighbors:
                if g.vs.find(var_name).degree() == 0:
                    remove_variable(self, var_name)
        
    
    # ----------------------- Variable node actions ---------
    def add_variable_node(self, name):
        self.__check_variable(name, exception=True)
        self.__create_variable_node(name)
    
    def remove_variable(self, name):
        self.__check_variable(name, exception=True, target='variable')
        if self._graph.vs.find(name).degree() == 0:
            self._graph.delete_vertices(self._graph.vs.find(name).index)
        else:
            raise Exception('Can not delete variables with degree >0')
            
    def __create_variable_node(self, name):
        self._graph.add_vertex(name)
        self._graph.vs.find(name=name)['is_factor'] = False
        
    # ----------------------- Info --------------------------
    def get_variable_status(self, name):
        return self.__check_variable(name)
    
    def __check_variable(self, name, exception=False, target=None):
        if len(self._graph.vs) == 0:
            return False
        elif len(self._graph.vs.select(name_eq=name)) == 0:
            return False
        else:
            if self._graph.vs.find(name=name)['is_factor'] == True:
                if (target == 'variable') and exception: raise Exception('Factor name. Variable expected')
                return 'factor'
            else:
                if (target == 'factor') and exception: raise Exception('Variable name. Factor expected')
                return 'variable'
    
    # ----------------------- Graph structure ---------------
    def is_connected(self):
        self._graph.is_connected()
        
    def is_tree(self):
        self._graph.is_tree()

#### Example

In [3]:
fg = factor_graph()
fg.add_factor_node('p1', factor(['x1', 'x2', 'x3']))
fg.add_factor_node('p2', factor(['x2', 'x4']))

### 1.2 Factor graph from string

In [4]:
def string2factor_graph(str_):
    res_factor_graph = factor_graph()
    
    str_ = [i.split('(') for i in str_.split(')') if i != '']
    for i in range(len(str_)):
        str_[i][1] = str_[i][1].split(',')
        
    for i in str_:
        res_factor_graph.add_factor_node(i[0], factor(i[1]))
    
    return res_factor_graph

#### Example

In [5]:
string2factor_graph('phi_1(a,b,c)phi_2(b,d,e)')

### 1.3 Factor graph visualization

In [6]:
def plot_factor_graph(x):
    graph = net.Network(notebook=True, width="100%")
    graph.toggle_physics(False)
    
    # Vertices
    label = x._graph.vs['name']
    color = ['#2E2E2E' if i is True else '#F2F2F2' for i in x._graph.vs['is_factor']]
    graph.add_nodes(range(len(x._graph.vs)), label=label, color=color)
    
    # Edges
    graph.add_edges(x._graph.get_edgelist())
    
    return graph.show("graph.html")

#### Example